In [1]:
import os
import shutil
import h5py
from eventlist_v5 import Eventlist
import matplotlib.pyplot as plt
import tables as tb
import numpy as np
from tqdm import tqdm
from scipy.signal import find_peaks
from astropy.time import Time, TimeDelta
plt.rcParams['figure.figsize'] = [25, 10]

In [2]:
# List of files in directory /data01
os.listdir('/data01')

['output',
 'wf_runId_00115_configId_00000_2022-10-03T22_39_08.295757.h5',
 'wf_runId_00240_configId_00000_2022-10-10T08_21_49.133517.h5',
 'wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.h5',
 'wf_runId_00261_configId_00000_2022-10-24T08_30_50.143648.h5']

In [63]:
h5_file_name = os.listdir('/data01')[3]

# __Test__
* DLO: `wf_runId_00115_configId_00000_2022-10-03T22_39_08.295757.h5`
    * single event: `dl0_id=18` and `dl1_id=18`
    * double event: `dl0_id=315` and `dl1_id=315`


In [64]:
dl1_path = os.path.join(os.path.dirname(os.getcwd()), 'dl1')
# Filepath for dl0 and dl1
filePath_dl0 = f"/data01/{h5_file_name}"
filePath_dl1 = os.path.join(os.path.join(dl1_path, 'output_dl1'), 
                            os.path.basename(filePath_dl0).replace('.h5', '.dl1.h5')) 
# Output dir for dl2
outputdir=os.path.join(dl1_path, 'output_dl2_abcd')
outputdir0_old=os.path.join(dl1_path, 'output_dl2_fromdl0_old')
outputdir0=os.path.join(dl1_path, 'output_dl2_fromdl0')
outputdir1=os.path.join(dl1_path, 'output_dl2_fromdl1')
print(filePath_dl0)
print(filePath_dl1)

/data01/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.h5
/home/usergamma/workspace/dl1/output_dl1/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl1.h5


In [65]:
cmd = f"python ../dl1/dl02dl1.py {filePath_dl0} {dl1_path}/output_dl1"
!echo $ {cmd}
!{cmd}

$ python ../dl1/dl02dl1.py /data01/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.h5 /home/usergamma/workspace/dl1/output_dl1
/home/usergamma/workspace/dl1/output_dl1/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl1.h5
100%|█████████████████████████████████████| 10000/10000 [02:50<00:00, 58.73it/s]


In [66]:
eventlist_dl0 = Eventlist()
eventlist_dl1 = Eventlist(from_dl1=True)

### __DL0 smaple example__

In [27]:
# if os.path.exists(outputdir):
#     shutil.rmtree(outputdir)
# eventlist_dl0.process_file(filePath_dl0, None, outputdir, True, 1565, 1565)

In [28]:
# if os.path.exists(outputdir):
#     shutil.rmtree(outputdir)
# eventlist_dl1.process_file(filePath_dl1, None, outputdir, True, 1585, 1585)

## __DL0 to DL2__

In [67]:
if os.path.exists(outputdir0):
    shutil.rmtree(outputdir0)
eventlist_dl0.process_file(filePath_dl0, None, outputdir0, False, 0, -1)

Processing /data01/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.h5
La directory '/home/usergamma/workspace/dl1/output_dl2_fromdl0' è stata creata.
Il file '/home/usergamma/workspace/dl1/output_dl2_fromdl0/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl2.h5' non esiste.


100%|██████████| 10000/10000 [01:13<00:00, 136.14it/s]


## __DL1 to DL2__

In [68]:
if os.path.exists(outputdir1):
    shutil.rmtree(outputdir1)
eventlist_dl1.process_file(filePath_dl1, None, outputdir1, False, 0, -1)

Processing /home/usergamma/workspace/dl1/output_dl1/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl1.h5
La directory '/home/usergamma/workspace/dl1/output_dl2_fromdl1' è stata creata.
Il file '/home/usergamma/workspace/dl1/output_dl2_fromdl1/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl1.dl2.h5' non esiste.


100%|██████████| 10158/10158 [00:58<00:00, 175.02it/s]


## __Check if the two DL2 file are the same__

In [69]:
filePath_dl0_old_dl2 = os.path.join(outputdir0_old, os.path.basename(filePath_dl0)).replace('h5', 'dl2.h5')
filePath_dl0_dl2     = os.path.join(outputdir0, os.path.basename(filePath_dl0)).replace('h5', 'dl2.h5')
filePath_dl1_dl2     = os.path.join(outputdir1, os.path.basename(filePath_dl1)).replace('h5', 'dl2.h5')
print(filePath_dl0_old_dl2)
print(filePath_dl0_dl2)
print(filePath_dl1_dl2)

/home/usergamma/workspace/dl1/output_dl2_fromdl0_old/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl2.h5
/home/usergamma/workspace/dl1/output_dl2_fromdl0/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl2.h5
/home/usergamma/workspace/dl1/output_dl2_fromdl1/wf_runId_00249_configId_00000_2022-10-17T09_45_54.710368.dl1.dl2.h5


### __DL2 of DL0 vs DL2 of DL1__

In [70]:
cnt_pk_notfound_h50 = 0
cnt_pk_notfound_h51 = 0
cnt_len_h50 = 0
cnt_len_h51 = 0
with h5py.File(filePath_dl0_dl2, mode='r') as h50:
    cnt_len_h50 = len(h50['dl2']['eventlist'])
    evlst0 = h50['dl2']['eventlist']
    for data0 in evlst0:
        if -1 in data0:
            cnt_pk_notfound_h50 += 1
with h5py.File(filePath_dl1_dl2, mode='r') as h51:
    cnt_len_h51 = len(h51['dl2']['eventlist'])
    evlst1 = h51['dl2']['eventlist']
    for data1 in evlst1:
        if -1 in data1:
            cnt_pk_notfound_h51 += 1

print(f"len h5_0=            {cnt_len_h50}")
print(f"len h5_1=            {cnt_len_h51}")
print(f"pks_not_found h5_0=  {cnt_pk_notfound_h50}")
print(f"pks_not_found h5_1=  {cnt_pk_notfound_h51}")

len h5_0=            10159
len h5_1=            10159
pks_not_found h5_0=  13
pks_not_found h5_1=  13


In [72]:
with h5py.File(filePath_dl0_dl2, mode='r') as h5_0:
    with h5py.File(filePath_dl1_dl2, mode='r') as h5_1:
        i, j = 0, 0
        evlst0 = h5_0['dl2']['eventlist']
        evlst1 = h5_1['dl2']['eventlist']
        for data0, data1 in tqdm(zip(evlst0, evlst1), total=cnt_len_h50):
            if not data0==data1:
                print('dl0_2_dl2:', data0)
                print('dl1_2_dl2:', data1)
                # print('dl0_2_dl2 mult:', data0['mult'])
                # print('dl1_2_dl2 mult:', data1['mult'])
                print()

  0%|          | 0/10159 [00:00<?, ?it/s]

 82%|████████▏ | 8337/10159 [00:02<00:00, 3520.43it/s]

dl0_2_dl2: (31., 1201., 19493.26, 19759.607, 16815.045, 0., 7720., 486., -300., 1.66600005e+09)
dl1_2_dl2: (31., 1201., 19493.26, 19849.68, 16833.98, 0., 7720., 486., -300., 1.66600005e+09)



100%|██████████| 10159/10159 [00:03<00:00, 3366.99it/s]

dl0_2_dl2: (33., 1200., 21430.01, 21790.027, 19032.541, 0., 9987., 558., -300., 1.66600008e+09)
dl1_2_dl2: (33., 1200., 21430.01, 21880.264, 19034.832, 0., 9987., 558., -300., 1.66600008e+09)



In [10]:
# with open(os.path.join(outputdir0_old, 'old_new___differences.txt'), 'w') as f_diff:
#     with h5py.File(filePath_dl0_old_dl2, mode='r') as h5_0:
#         with h5py.File(filePath_dl0_dl2, mode='r') as h5_1:
#             for data0, data1 in tqdm(zip(h5_0['dl2']['eventlist'], h5_1['dl2']['eventlist']), 
#                                     total=len(h5_1['dl2']['eventlist'])):
#                 if not data0==data1:
#                     f_diff.write(f'data0: {data0} data1: {data1}\n')

### __DL0 vs DL1__

In [54]:
# with h5py.File(filePath_dl0_dl2, mode='r') as h50:
#     with h5py.File(filePath_dl1_dl2, mode='r') as h51:
#         print(f"len h5_0={len(h50['dl2']['eventlist'])}")
#         print(f"len h5_1={len(h51['dl2']['eventlist'])}")